# サンプルコード集

## ソースコードからASTへの変換

astroidでPythonのソースコードをASTに変換する。

In [52]:
import astroid

code = """
( x + 1 ) * 2
"""
node = astroid.extract_node(code)
print(node.repr_tree())


BinOp(
   op='*',
   left=BinOp(
      op='+',
      left=Name(name='x'),
      right=Const(
         value=1,
         kind=None)),
   right=Const(
      value=2,
      kind=None))


## print関数を利用するコードの構造

`print("Hello World")` をASTに変換する。

In [53]:
import astroid

code = """
print("Hello World")
"""
node = astroid.extract_node(code)
print(node.repr_tree())

Call(
   func=Name(name='print'),
   args=[Const(
         value='Hello World',
         kind=None)],
   keywords=[])


関数ではなくメソッド呼び出しの場合はfunc属性がNameノードではなくAttributeノードになるので区別できる。

In [54]:
import astroid

code = """
obj.print("Hello World")
"""
node = astroid.extract_node(code)
print(node.repr_tree())

Call(
   func=Attribute(
      attrname='print',
      expr=Name(name='obj')),
   args=[Const(
         value='Hello World',
         kind=None)],
   keywords=[])


## print関数利用の判定ロジック

In [55]:
import astroid
from astroid import nodes


def is_print_function_call(node: nodes.NodeNG) -> bool:
    if not isinstance(node, nodes.Call):
        return False

    call_target = node.func
    if not isinstance(call_target, nodes.Name):
        return False

    return call_target.name == "print"


node1 = astroid.extract_node("""
print("Hello World")
""")
print(node1.as_string(), " is print function call ?")
print(is_print_function_call(node1))

node2 = astroid.extract_node("""
obj.print("Hello World")
""")
print(node2.as_string(), " is print function call ?")
print(is_print_function_call(node2))

node3 = astroid.extract_node("""
str("Hello World")
""")
print(node3.as_string(), " is print function call ?")
print(is_print_function_call(node3))


print('Hello World')  is print function call ?
True
obj.print('Hello World')  is print function call ?
False
str('Hello World')  is print function call ?
False


## print関数利用を検知するCheckerクラス

In [56]:
from astroid import nodes
from pylint.checkers import BaseChecker
from pylint.typing import MessageDefinitionTuple


def is_print_function_call(node: nodes.NodeNG) -> bool:
    if not isinstance(node, nodes.Call):
        return False

    call_target = node.func
    if not isinstance(call_target, nodes.Name):
        return False

    return call_target.name == "print"


class PrintFunctionChecker(BaseChecker):
    name = "print-function-checker"
    msgs: dict[str, MessageDefinitionTuple] = {
        "E9901": (
            "Used print function",
            "no-print-function",
            "print function should not be used.",
        ),
    }

    def visit_call(self, node: nodes.Call):
        if is_print_function_call(node):
            self.add_message("no-print-function", node=node)



## 別名が付けられたprint関数の利用を検知できるのか？

print2にprint関数を代入して呼び出すと、Callノードの関数名がprint2に変わってしまう。
Callノードの関数名を見ただけでは別名を見分けることができない。

In [57]:
import astroid

code = """
print2 = print
print2("検知できるかな？")
"""
node = astroid.extract_node(code)  # extract_nodeはデフォルトでコードの最終行をノードとして取り出す
print(node.repr_tree())

Call(
   func=Name(name='print2'),
   args=[Const(
         value='検知できるかな？',
         kind=None)],
   keywords=[])


## astroidのinference(推論)機能の紹介

推論前はcという変数名以外の情報を持っていない。
推論後はcにa + bを計算した結果の定数の3が代入されていることが分かる。

In [58]:
import astroid

code = """
a = 1
b = 2
c = a + b
c
"""
name_node_c = astroid.extract_node(code)
print("推論前のノード")
print(name_node_c.repr_tree())

print("推論後のノード")
inferred = next(name_node_c.infer())
print(inferred.repr_tree())

推論前のノード
Name(name='c')
推論後のノード
Const(
   value=3,
   kind=None)


## inferenceを使った関数呼び出しの名前解決

推論前のCallノードのfunc属性はprint2という名前のNameノード。
推論後はオリジナルの関数定義であるFunctionDefノードが得られ、名前もprintであることが分かる。
FunctionDefの親ノードも見るとbuiltinsという名前のモジュールであることが分かり、組み込みモジュールで定義されたprint関数であることが確実になる。

In [59]:
import astroid

code = """
print2 = print
print2("検知できるかな？")
"""
call_node = astroid.extract_node(code)
call_target = call_node.func
print("推論前の呼び出し対象の種類")
print(type(call_target))
print("推論前の呼び出し対象の名前")
print(call_target.name)

inferred = next(call_target.infer())
print("推論後の呼び出し対象の種類")
print(type(inferred))
print("推論後の呼び出し対象の名前")
print(inferred.name)

scope = inferred.parent
print("推論後の呼び出し対象のスコープ")
print(type(scope))
print("推論後の呼び出し対象のスコープ名")
print(scope.name)


推論前の呼び出し対象の種類
<class 'astroid.nodes.node_classes.Name'>
推論前の呼び出し対象の名前
print2
推論後の呼び出し対象の種類
<class 'astroid.nodes.scoped_nodes.scoped_nodes.FunctionDef'>
推論後の呼び出し対象の名前
print
推論後の呼び出し対象のスコープ
<class 'astroid.nodes.scoped_nodes.scoped_nodes.Module'>
推論後の呼び出し対象のスコープ名
builtins


## ネストしたif文をどうやって検知するか？

### 3重にネストしたif文

In [60]:
import astroid

code = """
if condition1:
    if condition2:
        if condition3:
            pass
"""
node = astroid.extract_node(code)
print(node.repr_tree())

If(
   test=Name(name='condition1'),
   body=[If(
         test=Name(name='condition2'),
         body=[If(
               test=Name(name='condition3'),
               body=[Pass()],
               orelse=[])],
         orelse=[])],
   orelse=[])



### 途中に別のノードが存在する場合もある

In [61]:
import astroid

code = """
if condition1:
    while condition2:
        if condition3:
            pass
"""
node = astroid.extract_node(code)
print(node.repr_tree())


If(
   test=Name(name='condition1'),
   body=[While(
         test=Name(name='condition2'),
         body=[If(
               test=Name(name='condition3'),
               body=[Pass()],
               orelse=[])],
         orelse=[])],
   orelse=[])


## ノードを辿る実装の課題

visit_ifメソッドが複雑になってしまっている。

In [62]:
from astroid import nodes
from pylint.checkers import BaseChecker

MAX_NEST_LEVEL = 3


class NestedIfChecker(BaseChecker):
    name = "nested-if-checker"
    msgs: dict[str, MessageDefinitionTuple] = {
        "E9902": (
            "if statements nested too much",
            "too-nested-if",
            "if statements should be nested too much",
        ),
    }

    def visit_if(self, node: nodes.If):
        nest_level = 1
        scope = node
        while scope := scope.parent:
            if isinstance(scope, nodes.If):
                if (nest_level := nest_level + 1) > MAX_NEST_LEVEL:
                    self.add_message("too-nested-if", node=node)
                    break

## スタックを利用したスコープ管理の実装

全体のコードの行数は増えたが、スタックの操作とネストの深さの判定で処理が分離できていて、判定ロジックの見通しが良い。

In [63]:
from astroid import nodes
from pylint.checkers import BaseChecker

MAX_NEST_LEVEL = 3


class NestedIfChecker(BaseChecker):
    name = "nested-if-checker"
    msgs: dict[str, MessageDefinitionTuple] = {
        "E9902": (
            "if statements nested too much",
            "too-nested-if",
            "if statements should be nested too much",
        ),
    }

    def open(self):
        self._if_stack: list[nodes.If] = []

    def visit_if(self, node: nodes.If):
        self._if_stack.append(node)

        if self.nest_level() > MAX_NEST_LEVEL:
            self.add_message("too-nested-if", node=node)

    def leave_if(self, node: nodes.If):
        self._if_stack.pop()

    def nest_level(self) -> int:
        return len(self._if_stack)

    def inside_if_block(self) -> bool:
        return self.nest_level() > 0